# Tutorial 3: Other Methods
In this tutorial, we'll describe how to incorporate other Lipschitz estimation techniques into our codebase. For each method, we try to use the official codebase attached to the papers. We simply built an interface on top of each of these to be more amenable to our system. The methods we consider are:
* **CLEVER**: Uses randomly sampled points and extremal value theory to generate an heuristic Lipschitz estimate([github](https://github.com/IBM/CLEVER-Robustness-Score)). 
* **FastLip**: Uses the hyperbox and boolean hyperbox abstract domains to efficiently generate an upper bound to the Lipschitz constant ([github](https://github.com/huanzhang12/CertifiedReLURobustness)).
* **LipLP**: The naive linear programming relaxation to LipMIP
* **LipSDP**: Uses incremental quadratic constraints and semidefinite programming to generate a global upper bound of Lipschitz constants in the $\ell_2$ setting ([github](https://github.com/arobey1/LipSDP)).
* **SeqLip**: Frames Lipschitz estimation as a combinatorial optimization problem and uses greedy methods to generate a heuristic Lipschitz estimate ([github](https://github.com/avirmaux/lipEstimation)).
* **RandomLB**: Randomly samples points and takes their maximal gradient norm. This is like CLEVER, but doesn't use the extremal value theory step, and thereby provides a certifiable lower bound.
* **NaiveUB**: Multiplies the operator norm of each component of a ReLU network together to yield an extremely loose upper bound.

Note that LipSDP uses the Mosek plugin for matlab to solve SDP's. Follow the instructions in their github to install these dependencies.

In [33]:
# Imports
import matlab.engine

import sys
sys.path.append('..')
import torch 
from pprint import pprint 

import utilities as utils
from relu_nets import ReLUNet
from hyperbox import Hyperbox 
from lipMIP import LipMIP
from other_methods import CLEVER, FastLip, LipLP, LipSDP, NaiveUB, RandomLB, SeqLip
import experiment as exp 

# 1: Individual Methods
The interface to run each method is identical, and all inherit a generic `OtherResult` class (except for LipMIP). We demonstrate how to run each method here.

Many methods have variants and hyperparameters that can be tuned. We incorporate these as kwargs and can tune them in our repository, but leave them mostly as default from their original codebases.

In [23]:
# Basic network example 
test_network = ReLUNet([2, 16, 16, 2])
test_domain = Hyperbox.build_unit_hypercube(2)
primal_norm = 'linf'
c_vector = torch.Tensor([1.0, -1.0])

In [24]:
# CLEVER (the first one alphabetically, so we'll use this as an example)
test_clever = CLEVER(test_network, c_vector, test_domain, primal_norm) # builds a CLEVER instance
clever_out = test_clever.compute() # executes CLEVER 
# Let's examine the output of CLEVER for a moment
print('Lipschitz estimate (as a float) is:', clever_out)

# We also store these attributes in the test_clever object (of class CLEVER)
print('test_clever holds the value (%f), and compute_time (%f s)' % (test_clever.value, test_clever.compute_time))


Lipschitz estimate (as a float) is: 0.2372237592935562
test_clever holds the value (0.237224), and compute_time (1.945315 s)


In [25]:
# Evaluating all the methods (using the same interface)
for other_method in [CLEVER, FastLip, LipLP, LipSDP, NaiveUB, RandomLB, SeqLip]:
    test_object = other_method(test_network, c_vector, domain=test_domain, primal_norm=primal_norm)
    test_object.compute()
    print(other_method.__name__ + ' ran in %.02f seconds and has value %.04f' % 
          (test_object.compute_time, test_object.value))

# CAVEAT! Some methods, such as LipSDP output an l2-lipschitz value, which needs to be scaled according to dimension

CLEVER ran in 1.81 seconds and has value 0.2372
FastLip ran in 0.00 seconds and has value 1.6836
LipLP ran in 0.01 seconds and has value 1.6212
LipSDP ran in 5.93 seconds and has value 0.6141
NaiveUB ran in 0.00 seconds and has value 6.7531
RandomLB ran in 0.42 seconds and has value 0.2372
SeqLip ran in 0.01 seconds and has value 0.9520


# 2: The `Experiment` class
As a convenient and flexible shorthand to evaluate lipschitz constants of various networks under various settings, we built the `Experiment` class which is very handy for performing common operations.

In [27]:
eval_class = [CLEVER, FastLip, LipLP, LipSDP, NaiveUB, RandomLB, SeqLip, LipMIP]

In [28]:
# Use case 1: evaluating local lipschitz constants for a fixed network, across many small random domains

# --- build experiment object 
basic_exp = exp.Experiment(eval_class, network=test_network, c_vector=c_vector, primal_norm=primal_norm)

# --- run all methods across 10 random hyperboxes, centered in [0,1]^2, with fixed radius 0.2
sample_domain = Hyperbox.build_unit_hypercube(2)
sample_factory = utils.Factory(Hyperbox.build_linf_ball, radius=0.2)
random_out = basic_exp.do_random_evals(3, sample_domain, sample_factory) # This should take about a minute

In [30]:
'''Examining random_out...
random_out is a ResultList object, which is a list-like wrapper for individual Result Objects
We'll first interact with a Result object which considers local lipschitz estimation for only one of the 3 
random points we evaluated above. ResultList allows us to collect average and standard deviations for values
and compute times
'''
result_0 = random_out.results[0] # first result 
result_0

In [36]:
# Just the values:
pprint(result_0.values())

{'CLEVER': 0.24502065777778625,
 'FastLip': 1.5437431,
 'LipLP': 1.4552422256126802,
 'LipMIP': 0.24502267002602715,
 'LipSDP': 0.6140934708628029,
 'NaiveUB': 6.7530985,
 'RandomLB': tensor(0.2450),
 'SeqLip': 0.9519979580714448}


In [37]:
# Just the compute times 
pprint(result_0.compute_times())

{'CLEVER': 1.897946834564209,
 'FastLip': 0.0011141300201416016,
 'LipLP': 0.01175236701965332,
 'LipMIP': 0.03613924980163574,
 'LipSDP': 5.903670310974121,
 'NaiveUB': 0.0002651214599609375,
 'RandomLB': 0.37348198890686035,
 'SeqLip': 0.006624460220336914}


In [38]:
# The entire objects
pprint(result_0.objects())

{'CLEVER': <other_methods.clever.CLEVER object at 0x7f030f4b7278>,
 'FastLip': <other_methods.fast_lip.FastLip object at 0x7f030f4b77f0>,
 'LipLP': <other_methods.lip_lp.LipLP object at 0x7f030f4b7908>,
 'LipMIP': LipMIP Result: 
	Value 0.245
	Runtime 0.036,
 'LipSDP': <other_methods.lip_sdp.LipSDP object at 0x7f030f4b7b38>,
 'NaiveUB': <other_methods.naive_methods.NaiveUB object at 0x7f030f4b74a8>,
 'RandomLB': <other_methods.naive_methods.RandomLB object at 0x7f030f4b7e80>,
 'SeqLip': <other_methods.seq_lip.SeqLip object at 0x7f030f4b7940>}


In [44]:
# To operate on a result_list, you can access mean and standard deviations directly:
print('values:')
pprint(random_out.average_stdevs('value'))
print('\ntimes:')
pprint(random_out.average_stdevs('time'))

# where each triple is of the form (average, stdev, n)

values:
{'CLEVER': (0.23219680289427438, 0.013017590767464945, 3),
 'FastLip': (1.1594396, 0.29563046, 3),
 'LipLP': (1.0838283144675767, 0.2828449739617439, 3),
 'LipMIP': (0.23219880482980879, 0.013017597637645817, 3),
 'LipSDP': (0.6140934708628029, 0.0, 3),
 'NaiveUB': (6.753099, 4.7683716e-07, 3),
 'RandomLB': (0.23219681, 0.013017589, 3),
 'SeqLip': (0.9519979580714448, 0.0, 3)}

times:
{'CLEVER': (1.9245862166086833, 0.06511572578781501, 3),
 'FastLip': (0.0012742678324381511, 0.0002159351453616695, 3),
 'LipLP': (0.011168082555135092, 0.0004139741795789362, 3),
 'LipMIP': (0.0366217295328776, 0.0037957176296600555, 3),
 'LipSDP': (5.911162694295247, 0.03566331163237689, 3),
 'NaiveUB': (0.00029889742533365887, 3.616443021838349e-05, 3),
 'RandomLB': (0.3862941265106201, 0.016450974104802892, 3),
 'SeqLip': (0.007067521413167317, 0.00033200517416576993, 3)}


In [45]:
# Use case 2: evaluating local lipschitz constants across [0,1]^d
basic_exp.do_l


In [ ]:
# Use case 3: evaluating local lipschitz constants across [-r, +r]^d where r is a parameter taken to be large
